0. Package Installation

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

import time

In [2]:
from utils.imageloader import load_images
from utils.dataloader import load_data
from utils.normalize import batch_normalize
from utils.gram_matrix import gram_matrix
from model.VGG16 import VGG16
from model.TransformerNet import TransformerNet

1. Hyperparameter Setting

In [3]:
batch_size = 10
num_epoch = 5
learning_rate = 1e-4
content_weight = 1e5
style_weight = 1e10
log_interval = 50
ckpt_dir = './checkpoints'

2. Style Images and Train Data Loading

In [4]:
style_data = load_images('./data/', 'summer', batch_size)
print(style_data.shape)

train_dataset, train_dataloader = load_data('./data/', batch_size)
print(train_dataset[0][0].shape)

torch.Size([100, 3, 256, 256])
torch.Size([3, 256, 256])


3. Style Transform with gram

In [5]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [6]:
transformer = TransformerNet().to(device)
vgg = VGG16(requires_grad=False).to(device)

features_style = vgg(batch_normalize(style_data.to(device)))
gram_style = [gram_matrix(y) for y in features_style]

C:\Users\jeongmin\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\jeongmin\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


4. TransformerNet training with train data

In [7]:
optimizer = optim.Adam(transformer.parameters(), lr=learning_rate)
loss_function = nn.MSELoss()

In [8]:
def train():
    for epoch in range(num_epoch):
        transformer.train()
        agg_content_loss = 0.
        agg_style_loss = 0.
        count = 0

        for batch_id, (x, _) in enumerate(train_dataloader):
            n_batch = len(x)
            count += n_batch
            optimizer.zero_grad()

            x = x.to(device)
            y = transformer(x)

            y = batch_normalize(y)
            x = batch_normalize(x)

            features_y = vgg(y)
            features_x = vgg(x)

            content_loss = content_weight * loss_function(features_y.relu2_2, features_x.relu2_2)

            style_loss = 0.
            for ft_y, gm_s in zip(features_y, gram_style):
                gm_y = gram_matrix(ft_y)
                style_loss += loss_function(gm_y, gm_s[:n_batch, :, :])
            style_loss *= style_weight

            total_loss = content_loss + style_loss
            total_loss.backward()
            optimizer.step()

            agg_content_loss += content_loss.item()
            agg_style_loss += style_loss.item()

            if (batch_id + 1) % log_interval == 0:
                msg = "{}\tEpoch {}:\t[{}/{}]\tcontent: {:.6f}\tstyle: {:.6f}\ttotal: {:.6f}".format(
                    time.ctime(), epoch + 1, count, len(train_dataset),
                    agg_content_loss / (batch_id + 1),
                    agg_style_loss / (batch_id + 1),
                    (agg_content_loss + agg_style_loss) / (batch_id + 1)
                )
                print(msg)
                
        # 4.1 Save Model
        transformer.eval().cpu()
        ckpt_model_filename = "ckpt_epoch_" + str(epoch) + "_batch_id_" + str(batch_id + 1) + ".pth"
        print(str(epoch), "th checkpoint is saved!")
        ckpt_model_path = os.path.join(ckpt_dir, ckpt_model_filename)
        torch.save({
            'epoch': epoch,
            'model_state_dict': transformer.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss
        }, ckpt_model_path)

        transformer.to(device).train()

In [9]:
train()

Fri Dec  2 19:21:31 2022	Epoch 1:	[500/2793]	content: 632744.739375	style: 5391781.840000	total: 6024526.579375
Fri Dec  2 19:22:19 2022	Epoch 1:	[1000/2793]	content: 639149.101562	style: 5059366.335000	total: 5698515.436562
Fri Dec  2 19:23:07 2022	Epoch 1:	[1500/2793]	content: 649858.648958	style: 4785013.368333	total: 5434872.017292
Fri Dec  2 19:23:55 2022	Epoch 1:	[2000/2793]	content: 654773.149844	style: 4577643.715000	total: 5232416.864844
Fri Dec  2 19:24:43 2022	Epoch 1:	[2500/2793]	content: 663434.472375	style: 4396012.344000	total: 5059446.816375
0 th checkpoint is saved!
Fri Dec  2 19:25:57 2022	Epoch 2:	[500/2793]	content: 694711.118750	style: 3370244.495000	total: 4064955.613750
Fri Dec  2 19:26:43 2022	Epoch 2:	[1000/2793]	content: 690293.465000	style: 3302690.722500	total: 3992984.187500
Fri Dec  2 19:27:28 2022	Epoch 2:	[1500/2793]	content: 687804.455417	style: 3240988.913333	total: 3928793.368750
Fri Dec  2 19:28:14 2022	Epoch 2:	[2000/2793]	content: 683488.900937	sty

5. Test